In [ ]:
from instance_pyfile import *
import random

In [ ]:
def parent2str(par1: Individual, par2: Individual) :
    outString = '{'

    outString += ('parent1 = ' + str(par1.trace) + ', parent2 = ' + str(par2.trace) )

    return outString

In [ ]:
def cut2Offsprings(llmResponse: str) :
    
    child1 = llmResponse[(llmResponse.find("<Off1>")+6) : (llmResponse.find("</Off1>"))]

    child2 = llmResponse[(llmResponse.find("<Off2>")+6) : (llmResponse.find("</Off2>"))]

    return child1, child2

In [ ]:
def cutMutated(llmResponse: str) :
    
    resStr = llmResponse[(llmResponse.find("<mut>")+5) : (llmResponse.find("</mut>"))]

    return resStr

In [ ]:
def OX(parent1, parent2):
    prompt = '{parent1 = ' + f"{parent1}, parent2 = {parent2}" + '} '

    size=len(parent1)
    # Step 1: Select random crossover range
    start, end = sorted(random.sample(range(1, size-2), 2))   # Avoid the first and last gene

    # Step 2: Create proto-child by inheriting the selected range from its parent
    child1 = [None]*len(parent1)
    child1[start:end] = parent1[start:end]

    child2 = [None]*len(parent2)
    child2[start:end] = parent2[start:end]
    
    loc1 = child1.copy()
    loc2 = child2.copy()

    # Step 3: Fill in remaining positions in child1 with other cities in parent2
    for i in range(len(parent2)):
        if parent2[i] not in child1:
            child1[child1.index(None)] = parent2[i]

    #Step 4: Fill in remaining positions in child2 with other cities in parent1
    for i in range(len(parent1)):
        if parent1[i] not in child2:
            child2[child2.index(None)] = parent1[i]

    prompt += ('{child1: ' + f"{child1}, child2: {child2}" + '} ' + 'Explanation: {Step 1: Select random crossover range in both 2 parents: ' + f"(start={start}, end={end}); ")
    prompt += ('Step 2: Create proto-child by inheriting the selected range from its parent: ' + f"(child1={loc1}, child2={loc2}); ")
    prompt += ('Step 3: Fill in remaining positions in child1 with other cities in parent2: ' + f"child1={child1}; ")
    prompt += ('Step 4: Fill in remaining positions in child2 with other cities in parent1: ' + f"child2={child2}" + '}\n')
    
    print(prompt)

In [ ]:
parent1 = list(random.sample(range(1, n + 1), n))
parent2 = list(random.sample(range(1, n + 1), n))

OX(parent1, parent2)

In [ ]:
def PMX(parent1, parent2):
    prompt = '{parent1 = ' + f"{parent1}, parent2 = {parent2}" + '} '

    size = len(parent1)
    # Step 1: Select crossover range at random
    start, end = sorted(random.sample(range(1, size - 2), 2))  # Avoid the first and last gene (the hive) 

    # Step 2: Create offspring by exchanging the selected range
    child1 = parent1[:start] + parent2[start:end] + parent1[end:]
    child2 = parent2[:start] + parent1[start:end] + parent2[end:]
    
    loc1 = child1.copy()
    loc2 = child2.copy()

    # Step 3: Determine the mapping relationship to legalize offspring
    mapping1 = {parent2[i]: parent1[i] for i in range(start, end)}
    mapping2 = {parent1[i]: parent2[i] for i in range(start, end)}

    # Step 4: Legalize children with the mapping relationship
    for i in list(range(start)) + list(range(end, size)):
        if child1[i] in mapping1: 
            while child1[i] in mapping1:
                child1[i] = mapping1[child1[i]]
        if child2[i] in mapping2:
            while child2[i] in mapping2:
                child2[i] = mapping2[child2[i]]

    prompt += ('{child1: ' + f"{child1}, child2: {child2}" + '} ' + 'Explanation: {Step 1: Select random crossover range in both 2 parents: ' + f"(start={start}, end={end}); ")
    prompt += ('Step 2: Create offspring by exchanging the selected range: ' + f"(child1={loc1}, child2={loc2}); ")
    prompt += ('Step 3: Determine the mapping relationship to legalize offspring: ' + f"(mapping1={mapping1}, mapping2={mapping2}); ")
    prompt += ('Step 4: Legalize children with the mapping relationsip and receive 2 result offsprings as shown.}\n')
    
    print(prompt)

In [ ]:
parent1 = list(random.sample(range(1, n + 1), n))
parent2 = list(random.sample(range(1, n + 1), n))

PMX(parent1, parent2)